In [ ]:
pip install colorama


In [ ]:
import random
import collections
import numpy as np
import pandas as pd
from collections import defaultdict
from prettytable import PrettyTable
from colorama import Fore, Style, init


'''
we're gonna have to define schedules or somethign like that, global variables for input

INPUT DATA:
Input data for each exam are teachers’ names, students ‘name, exam duration, courses
(course codes), and list of allowed classrooms (e.g. C301 to C310)

OUTPUT DATA:
Output data are classroom and starting time for each exam along with course code and
invigilating teacher. Time is determined by day (Monday to Friday) and start hour of the exam.


'''

 #======================= Sort of initlizting ===========================================
classRooms = [("C-301", 0), ("C-302", 1), ("C-303", 2), ("C-304", 3), ("C-305", 4),
              ("C-306", 5), ("C-307", 6), ("C-308", 7), ("C-309", 8), ("C-310", 9)]
totalClassRooms = len(classRooms)
classRoomCapacity = 30

# Weekdays only (Monday-Friday)
days = [("Monday", 0), ("Tuesday", 1), ("Wednesday", 2), ("Thursday", 3), ("Friday", 4)]
totalDays = len(days)

examStartTimings = [(9, 0), (14, 1)]  # 9 AM and 2 PM
totalExamStartTimings = len(examStartTimings)
examDuration = 3  # 3 hours per exam



# Genetic algorithm parameters
POPULATION_SIZE = 100
GENERATIONS = 50
CROSSOVER_PROB = 0.85
MUTATION_PROB = 0.15
ELITISM_COUNT = 20



# Penalty weights
PENALTY_WEIGHTS = {
    'missing_exam': 1000,
    'student_conflict': 500,
    'teacher_conflict': 300,
    'room_conflict': 300,
    'consecutive_teacher': 200,
    'weekend_exam': 1000,
    'invalid_time': 1000
}

# Reward weights
REWARD_WEIGHTS = {
    'friday_break': 50,
    'no_back_to_back': 30,
    'mg_before_cs': 40,
    'faculty_meeting_slot': 60
}

# Creating classes,  # These classes organize and store data about courses, teachers, students, and exams, making it easier to manage and manipulate the schedule using a Genetic Algorithm.
class Course:
    def __init__(self, code, name, num):
        self.code = code
        self.name = name
        self.num = num
        self.type = "MG" if "MG" in code else "CS" if "CS" in code else "OTHER"

class Registration:
    def __init__(self, name, course_codes):
        self.studentName = name
        self.registeredCourses = course_codes.copy()

class Exam:
    def __init__(self, course, teachers, classroom, day, start_time):
        self.course = course
        self.teachers = teachers if isinstance(teachers, list) else [teachers]
        self.classroom = classroom
        self.day = day
        self.start_time = start_time

# Define Individual namedtuple
Individual = collections.namedtuple('Individual', 'chromosome fitness constraints')

#Import CSV files and read them
def load_data():
    # Load courses
    courses_df = pd.read_csv('courses.csv')
    courses = [Course(row['Course Code'], row['Course Name'], idx)
               for idx, row in courses_df.iterrows()]

    # Load teachers
    teachers_df = pd.read_csv('teachers.csv')
    teachers = [row['Names'] for _, row in teachers_df.iterrows()]

    # Load students
    students_df = pd.read_csv('studentNames.csv')
    students = [row['Names'] for _, row in students_df.iterrows()]

    # Load registrations
    reg_df = pd.read_csv('studentCourse.csv')
    registrations = []
    for _, row in reg_df.iterrows():
        student = row['Student Name']
        course = row['Course Code']
        # Find or create registration
        found = False
        for reg in registrations:
            if reg.studentName == student:
                reg.registeredCourses.append(course)
                found = True
                break
        if not found:
            registrations.append(Registration(student, [course]))

    return courses, teachers, students, registrations

# Fitness evaluation
def evaluate_fitness(chromosome, registrations, MG_indexes, CS_indexes, courses, teachers):
    penalty = 0
    reward = 0

    #checking or making hard and soft constraints
    constraints = {
        'hard': {
            'all_courses_scheduled': True,
            'no_student_conflicts': True,
            'weekday_only': True,
            'valid_exam_times': True,
            'teacher_no_conflicts': True,
            'no_consecutive_teacher_assignments': True
        },
        'soft': {
            'friday_break': False,
            'no_back_to_back_exams': False,
            'mg_before_cs': False,
            'faculty_meeting_slots': False
        }
    }

    # 1. Check all courses are scheduled (HARD)
    scheduled_courses = {exam.course.code for exam in chromosome}
    all_courses = {course.code for course in courses}
    if scheduled_courses != all_courses:
        penalty += PENALTY_WEIGHTS['missing_exam']
        constraints['hard']['all_courses_scheduled'] = False

    # 2. Teacher assignments tracking (HARD)
    teacher_assignments = defaultdict(list)
    for exam in chromosome:
        time_slot = (exam.day[1], exam.start_time[1])
        for teacher in exam.teachers:
            teacher_assignments[teacher].append(time_slot)

    # Check for teacher conflicts
    for teacher, slots in teacher_assignments.items():
        if len(slots) != len(set(slots)):
            penalty += PENALTY_WEIGHTS['teacher_conflict']
            constraints['hard']['teacher_no_conflicts'] = False

    # Check for consecutive assignments (HARD)
    for teacher, slots in teacher_assignments.items():
        slots_sorted = sorted(slots)
        for i in range(1, len(slots_sorted)):
            prev_day, prev_time = slots_sorted[i-1]
            curr_day, curr_time = slots_sorted[i]
            if prev_day == curr_day and curr_time - prev_time < 2:
                penalty += PENALTY_WEIGHTS['consecutive_teacher']
                constraints['hard']['no_consecutive_teacher_assignments'] = False

    # 3. Room assignments tracking (HARD)
    room_assignments = defaultdict(set)
    for exam in chromosome:
        time_slot = (exam.day[1], exam.start_time[1])
        if time_slot in room_assignments[exam.classroom]:
            penalty += PENALTY_WEIGHTS['room_conflict']
            constraints['hard']['teacher_no_conflicts'] = False
        room_assignments[exam.classroom].add(time_slot)

    # 4. Student conflicts (HARD)
    student_schedule = defaultdict(set)
    for student in registrations:
        for exam_idx, exam in enumerate(chromosome):
            if exam.course.code in student.registeredCourses:
                time_slot = (exam.day[1], exam.start_time[1])
                if time_slot in student_schedule[student.studentName]:
                    penalty += PENALTY_WEIGHTS['student_conflict']
                    constraints['hard']['no_student_conflicts'] = False
                student_schedule[student.studentName].add(time_slot)

    # 5. Weekday and time checks (HARD)
    for exam in chromosome:
        if exam.day[1] >= 5:  # Weekend check
            penalty += PENALTY_WEIGHTS['weekend_exam']
            constraints['hard']['weekday_only'] = False
        if exam.start_time[0] not in {9, 14}:
            penalty += PENALTY_WEIGHTS['invalid_time']
            constraints['hard']['valid_exam_times'] = False

    # SOFT CONSTRAINT: Friday break (1-2 PM)
    friday_break_free = True
    for exam in chromosome:
        if exam.day[0] == "Friday" and exam.start_time[0] == 13:
            friday_break_free = False
            break
    if friday_break_free:
        reward += REWARD_WEIGHTS['friday_break']
        constraints['soft']['friday_break'] = True

    # SOFT CONSTRAINT: No back-to-back exams
    back_to_back_violations = 0
    for student in registrations:
        exam_times = []
        for exam_idx, exam in enumerate(chromosome):
            if exam.course.code in student.registeredCourses:
                exam_times.append((exam.day[1], exam.start_time[1]))

        exam_times_sorted = sorted(exam_times)
        for i in range(1, len(exam_times_sorted)):
            prev_day, prev_time = exam_times_sorted[i-1]
            curr_day, curr_time = exam_times_sorted[i]
            if prev_day == curr_day and curr_time - prev_time == 1:
                back_to_back_violations += 1

    if back_to_back_violations == 0:
        reward += REWARD_WEIGHTS['no_back_to_back']
        constraints['soft']['no_back_to_back_exams'] = True

    # SOFT CONSTRAINT: MG before CS
    mg_cs_violations = 0
    for student in registrations:
        mg_exam = None
        cs_exam = None
        for exam_idx, exam in enumerate(chromosome):
            if exam.course.code in student.registeredCourses:
                if exam.course.type == "MG":
                    mg_exam = exam
                elif exam.course.type == "CS":
                    cs_exam = exam

        if mg_exam and cs_exam:
            if (cs_exam.day[1] < mg_exam.day[1] or
               (cs_exam.day[1] == mg_exam.day[1] and
                cs_exam.start_time[1] < mg_exam.start_time[1])):
                mg_cs_violations += 1

    if mg_cs_violations == 0:
        reward += REWARD_WEIGHTS['mg_before_cs']
        constraints['soft']['mg_before_cs'] = True

    # SOFT CONSTRAINT: Faculty meeting slots
    faculty_schedule = defaultdict(set)
    for exam in chromosome:
        for teacher in exam.teachers:
            faculty_schedule[teacher].add((exam.day[1], exam.start_time[1]))

    # Check if at least half of faculty have a free slot on one day
    # and the other half on another day
    free_slots = defaultdict(set)
    for teacher in teachers:
        scheduled_slots = faculty_schedule.get(teacher, set())
        all_slots = {(day, time) for day in range(5) for time in [0, 1]}
        free_slots[teacher] = all_slots - scheduled_slots

    # Try to find two days where faculty can be split for meetings
    meeting_days_found = 0
    for day in range(5):
        day_free_count = sum(1 for teacher in teachers if any(slot[0] == day for slot in free_slots[teacher]))
        if day_free_count >= len(teachers) / 2:
            meeting_days_found += 1

    if meeting_days_found >= 2:
        reward += REWARD_WEIGHTS['faculty_meeting_slot']
        constraints['soft']['faculty_meeting_slots'] = True

    # Calculate final fitness
    fitness = reward / (1 + penalty) if penalty > 0 else reward + 1000
    return fitness, constraints

# Generate initial population of random schedules
def generate_population(size, courses, teachers, totalTeachers):
    return [Individual(
        chromosome=[random_exam(i, courses, teachers, totalTeachers)
                   for i in range(len(courses))],
        fitness=-1,
        constraints=None
    ) for _ in range(size)]

def random_exam(course_idx, courses, teachers, totalTeachers):
    """Generate a random exam schedule for a course"""
    course = courses[course_idx]

    # Assign random time slot (weekdays only, 9AM or 2PM)
    day = random.choice(days)
    time_slot = random.choice(examStartTimings)

    # Assign random invigilator(s) - 1-2 teachers per exam
    num_teachers = min(2, max(1, random.randint(1, 2)))
    invigilators = random.sample(teachers, num_teachers)

    # Assign random classroom
    classroom = random.choice(classRooms)

    return Exam(course, invigilators, classroom, day, time_slot)

#Mutation, Randomly modify parts of the schedule
def mutate(chromosome, courses, teachers, totalTeachers, mutation_prob):

    if random.random() < mutation_prob:
        idx = random.randint(0, len(chromosome) - 1)
        chromosome[idx] = random_exam(idx, courses, teachers, totalTeachers)
    return chromosome

#Crossover, Combine two schedules to produce offspring
def crossover(parent1, parent2, crossover_prob, method="one-point"):
    if random.random() > crossover_prob:
        return parent1, parent2  # No crossover

    size = len(parent1.chromosome)

    if method == "one-point":
        # One-point crossover: choose a random crossover point
        point = random.randint(1, size - 1)
        child1_chromosome = parent1.chromosome[:point] + parent2.chromosome[point:]
        child2_chromosome = parent2.chromosome[:point] + parent1.chromosome[point:]

    elif method == "two-point":
        # Two-point crossover: choose two points and swap the section between them
        point1, point2 = sorted(random.sample(range(1, size), 2))
        child1_chromosome = (parent1.chromosome[:point1] +
                             parent2.chromosome[point1:point2] +
                             parent1.chromosome[point2:])
        child2_chromosome = (parent2.chromosome[:point1] +
                             parent1.chromosome[point1:point2] +
                             parent2.chromosome[point2:])

    else:
        raise ValueError("Invalid crossover method. Choose 'one-point' or 'two-point'.")

    return Individual(child1_chromosome, -1, None), Individual(child2_chromosome, -1, None)


#selection function, Select individuals using fitness-proportional roulette wheel selection
def roulette_wheel_selection(population):

    # Calculate total fitness of positive values only
    total_fitness = sum(max(0, ind.fitness) for ind in population)

    # Handle case where all fitnesses are zero or negative
    if total_fitness <= 0:
        return random.choices(population, k=len(population))

    # Calculate selection probabilities
    selection_probs = [max(0, ind.fitness)/total_fitness for ind in population]

    # Select individuals
    selected = random.choices(population, weights=selection_probs, k=len(population))
    return selected


# ======================== VISUALIZATION & OUTPUT ========================
def print_generation_stats(generation, population):
    """Print statistics for the current generation"""
    print(f"\n{Fore.CYAN}=== GENERATION {generation + 1} ==={Style.RESET_ALL}")

    # Sort population by fitness
    sorted_pop = sorted(population, key=lambda x: x.fitness, reverse=True)

    # Print top 3 individuals
    for i in range(min(3, len(sorted_pop))):
        ind = sorted_pop[i]
        print(f"{Fore.YELLOW}Top {i+1}: Fitness = {ind.fitness:.2f}{Style.RESET_ALL}")

    # Print average fitness
    avg_fitness = sum(ind.fitness for ind in population) / len(population)
    print(f"{Fore.GREEN}Average fitness: {avg_fitness:.2f}{Style.RESET_ALL}")

def print_final_schedule(best_individual):
    """Print the final schedule in a formatted table"""
    print(f"\n{Fore.CYAN}=== BEST SCHEDULE FOUND ==={Style.RESET_ALL}")
    print(f"Fitness Score: {best_individual.fitness:.2f}")

    # Create schedule table
    schedule_table = PrettyTable()
    schedule_table.field_names = [
        "Course", "Day", "Time", "Room", "Teachers"
    ]

    # Color coding for different days
    day_colors = {
        "Monday": Fore.BLUE,
        "Tuesday": Fore.GREEN,
        "Wednesday": Fore.MAGENTA,
        "Thursday": Fore.YELLOW,
        "Friday": Fore.RED
    }

    for exam in best_individual.chromosome:
        day_color = day_colors.get(exam.day[0], Fore.WHITE)
        schedule_table.add_row([
            f"{exam.course.code}",
            f"{day_color}{exam.day[0]}{Style.RESET_ALL}",
            f"{exam.start_time[0]}:00",
            f"{exam.classroom[0]}",
            ", ".join(exam.teachers[:2]) + ("..." if len(exam.teachers) > 2 else "")
        ])

    print("\nFULL SCHEDULE:")
    print(schedule_table)

    # Print by day and time
    print(f"\n{Fore.CYAN}=== SCHEDULE BY DAY/TIME ==={Style.RESET_ALL}")
    schedule_by_time = defaultdict(list)
    for exam in best_individual.chromosome:
        key = (exam.day[0], exam.start_time[0])
        schedule_by_time[key].append(exam)

    for time_slot in sorted(schedule_by_time.keys()):
        day, hour = time_slot
        day_color = day_colors.get(day, Fore.WHITE)
        print(f"\n{day_color}{day} {hour}:00{Style.RESET_ALL}")
        for exam in schedule_by_time[time_slot]:
            print(f"  - {exam.course.code} in {exam.classroom[0]} "
                  f"(Teachers: {', '.join(exam.teachers[:2])})")

# ======================== MAIN ALGORITHM ========================
def genetic_algorithm():
    # Load dataset
    courses, teachers, students, registrations = load_data()
    totalTeachers = len(teachers)

    # Identify MG and CS courses
    MG_indexes = [i for i, c in enumerate(courses) if c.type == "MG"]
    CS_indexes = [i for i, c in enumerate(courses) if c.type == "CS"]

    # Initialize population
    population = generate_population(POPULATION_SIZE, courses, teachers, totalTeachers)

    # Evaluate initial population
    population = [
        ind._replace(
            fitness=evaluate_fitness(ind.chromosome, registrations, MG_indexes, CS_indexes, courses, teachers)[0],
            constraints=evaluate_fitness(ind.chromosome, registrations, MG_indexes, CS_indexes, courses, teachers)[1]
        )
        for ind in population
    ]

    # Evolution loop
    for generation in range(GENERATIONS):
        # Sort by fitness (elitism)
        population.sort(key=lambda x: x.fitness, reverse=True)

        # Print generation stats
        print_generation_stats(generation, population)

        # Selection
        selected = roulette_wheel_selection(population)

        # Crossover
        offspring = []
        for i in range(0, len(selected) - 1, 2):
            child1, child2 = crossover(selected[i], selected[i+1], CROSSOVER_PROB, method="one-point")
            offspring.extend([child1, child2])

        # Mutation
        offspring = [
            ind._replace(
                chromosome=mutate(ind.chromosome, courses, teachers, totalTeachers, MUTATION_PROB)
            )
            for ind in offspring
        ]

        # Evaluate offspring
        offspring = [
            ind._replace(
                fitness=evaluate_fitness(ind.chromosome, registrations, MG_indexes, CS_indexes, courses, teachers)[0],
                constraints=evaluate_fitness(ind.chromosome, registrations, MG_indexes, CS_indexes, courses, teachers)[1]
            )
            for ind in offspring
        ]

        # Create new generation (elitism + offspring)
        new_population = population[:ELITISM_COUNT] + offspring
        new_population = new_population[:POPULATION_SIZE]  # Maintain population size

        # Update population
        population = new_population

    # Return best solution
    best_solution = max(population, key=lambda x: x.fitness)
    return best_solution

# ======================== EXECUTION ========================
if __name__ == "__main__":
    best_schedule = genetic_algorithm()
    print_final_schedule(best_schedule)


=== GENERATION 1 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.01
Top 3: Fitness = 0.01
Average fitness: 0.01

=== GENERATION 2 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 3 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 4 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 5 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 6 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 7 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 8 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.01

=== GENERATION 9 ===
Top 1: Fitness = 0.02
Top 2: Fitness = 0.02
Top 3: Fitness = 0.02
Average fitness: 0.02

=== GENER